In [1]:
from fastai.tabular import *

In [2]:
path=Path('data/hotel')

In [3]:
path.ls()

[PosixPath('data/hotel/hotel_bookings.csv'),
 PosixPath('data/hotel/.ipynb_checkpoints'),
 PosixPath('data/hotel/models')]

In [4]:
train_df=pd.read_csv(path/'hotel_bookings.csv')

In [5]:
procs=[FillMissing, Categorify, Normalize]

In [6]:
cat_vars=['hotel', 'is_canceled','arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date']

          
          
cont_vars=['lead_time']        

In [7]:
dep_var = 'adr'
df = train_df[cat_vars + cont_vars + [dep_var]].copy()

In [16]:
train_df=None
gc.collect()

189

In [9]:
len(df)

119390

In [10]:
valid_idx=range(110000,119390)

In [11]:
len(valid_idx)

9390

In [12]:
data = (TabularList.from_df(df, path=path, cat_names=cat_vars, cont_names=cont_vars, procs=procs,)
                .split_by_idx(valid_idx)
                .label_from_df(cols=dep_var, label_cls=FloatList)
                .databunch())

In [13]:
data.batch_size=16

In [17]:
learn = tabular_learner(data, layers=[1000,500], ps=[0.001,0.01], emb_drop=0.04, metrics=rmse)

In [18]:
learn.fit_one_cycle(5,1e-3,wd=0.2)

epoch,train_loss,valid_loss,root_mean_squared_error,time
0,479.904877,504.376404,20.714249,01:49
1,490.664307,648.222717,23.360935,01:49
2,405.501068,496.118164,20.305229,01:51
3,398.719635,434.052979,18.790817,01:52
4,302.441437,487.367371,19.863329,01:47
